In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import random

import dotenv
from vllm import LLM, SamplingParams

from llm_ol.experiments.llm.templates import PROMPT_TEMPLATE, PROMPT_TEMPLATE_FULL
from llm_ol.dataset import data_model

dotenv.load_dotenv()

In [ ]:
# llm = LLM(model="out/experiments/finetune/v6/train/checkpoint-16500/merged")
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = llm.get_tokenizer()

In [ ]:
def generate(title, abstract):
    messages = [
        {
            "role": "user",
            "content": PROMPT_TEMPLATE_FULL.render(
                title=title, abstract=abstract, examples=[]
            ),
        }
    ]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    outputs = llm.generate(
        prompt,
        sampling_params=SamplingParams(
            temperature=0.1,
            top_p=0.9,
            max_tokens=1024,
            stop=["\n\n"],
        ),
    )
    return outputs[0].outputs[0].text

In [ ]:
generate(
    "A new way to generate electricity",
    "A new way to generate electricity has been discovered.",
)

In [ ]:
G = data_model.load_graph("out/data/arxiv/v1/full/full_graph.json")

In [ ]:
node = random.choice(list(G.nodes()))
page = G.nodes[node]["pages"][0]
print(page["title"])
print(page["abstract"])

out = generate(page["title"], page["abstract"])
print(out)